# imports

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

# constants

In [2]:
BATCH_SIZE = 128
EPOCHS = 6
LEARNING_RATE = 0.001
NEURONS = 128

# fetch

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

df = tfds.as_dataframe(ds_test, ds_info)
df.head()

2025-03-19 20:42:03.070724: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-03-19 20:42:03.640749: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,image,label
0,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",2
1,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",0
2,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",4
3,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",8
4,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",7


# prepare

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

# train

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_test,
)

Epoch 1/6


/Users/jtucker/Desktop/working2/keras-by-example/.environment/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6011 - sparse_categorical_accuracy: 0.8374 - val_loss: 0.1864 - val_sparse_categorical_accuracy: 0.9475
Epoch 2/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1716 - sparse_categorical_accuracy: 0.9505 - val_loss: 0.1375 - val_sparse_categorical_accuracy: 0.9583
Epoch 3/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1223 - sparse_categorical_accuracy: 0.9654 - val_loss: 0.1094 - val_sparse_categorical_accuracy: 0.9670
Epoch 4/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0960 - sparse_categorical_accuracy: 0.9720 - val_loss: 0.1015 - val_sparse_categorical_accuracy: 0.9703
Epoch 5/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0733 - sparse_categorical_accuracy: 0.9794 - val_loss: 0.0849 - val_sparse_categorical_accuracy: 0.9744
Epoch 6/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0579 - sparse_categorical_accuracy: 0.9841 - val_loss: 0.0839 - val_sparse_categorical_accuracy: 0.9748


# predict

In [6]:
predictions = model.predict(
    ds_test,
    batch_size=BATCH_SIZE,
    verbose=0,
)
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.001502,2.653304e-03,0.999991,0.252168,0.174911,0.025782,1.455435e-02,0.000185,0.099822,0.000438
1,1.000000,2.016279e-07,0.012043,0.000579,0.000096,0.641573,9.458644e-01,0.011427,0.271400,0.000155
2,0.000113,1.042062e-04,0.001097,0.025531,0.999998,0.000647,2.566294e-04,0.584064,0.040326,0.950858
3,0.350496,7.623140e-02,0.037630,0.304664,0.000125,0.074052,2.999558e-02,0.017199,0.998563,0.107042
4,0.019664,1.644276e-03,0.634081,0.616142,0.001101,0.000588,1.330418e-07,0.999998,0.048821,0.382237
